In [2]:
import numpy as np
import pandas as pd
import torch
import os
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import Sampler
import clip
from PIL import Image
from pathlib import Path
from tqdm.auto import tqdm
import re
from clip.simple_tokenizer import SimpleTokenizer
import faiss
import matplotlib.pyplot as plt
import random
#threshold for classification
thresh=0

_tokenizer = SimpleTokenizer()

# Copied from https://github.com/openai/CLIP/blob/beba48f35392a73c6c47ae67ddffced81ad1916d/clip/clip.py#L164
# but with relaxed exception
def tokenize(texts, context_length: int = 77) -> torch.LongTensor:
    if isinstance(texts, str):
        texts = [texts]

    sot_token = _tokenizer.encoder["<|startoftext|>"]
    eot_token = _tokenizer.encoder["<|endoftext|>"]
    all_tokens = [[sot_token] + _tokenizer.encode(text) + [eot_token] for text in texts]
    result = torch.zeros(len(all_tokens), context_length, dtype=torch.long)

    for i, tokens in enumerate(all_tokens):
        n = min(len(tokens), context_length)
        result[i, :n] = torch.tensor(tokens)[:n]
        if len(tokens) > context_length:
            result[i, -1] = tokens[-1]

    return result

# Load CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
#label csv
from sklearn.model_selection import train_test_split
df_data = pd.read_csv('/home/hice1/mmoradi6/scratch/RCLIP/data/train.csv', index_col='id')
df_labels=pd.read_csv('/home/hice1/mmoradi6/scratch/RCLIP/data/labels.csv',index_col="attribute_id")
image_dir="/home/hice1/mmoradi6/scratch/RCLIP/data/training_data/"
image_names=sorted(os.listdir(image_dir))[11000:21000]
train_names, test_names, _, _ = train_test_split(image_names, image_names, test_size=0.1, random_state=42)
text_list=[df_labels.iloc[idx].iloc[0].split("::")[1] for idx in range(len(df_labels))] #all the text as list
image_list=[preprocess(Image.open(image_dir+iii)).unsqueeze(0).to(device) for iii in test_names]
text=tokenize(text_list).to(device)


state_dir="/home/hice1/mmoradi6/scratch/RCLIP/models/"
states=sorted(os.listdir(state_dir))
with torch.no_grad():
    for i in states:
        #for each model
        TP=[]
        TN=[]
        FP=[]
        FN=[]
        state_path=state_dir+i
        state_dict=torch.load(state_path, map_location=device)
        model.load_state_dict(state_dict)
        model.eval()
        #get the probs
        for img_idx in range(len(image_list)):
            img=image_list[img_idx]
            logits_per_image, _ = model(image_list[0], text)
            probs = logits_per_image.softmax(dim=-1).detach().cpu().numpy()
            probs[0][probs[0]<=thresh]=0
            probs[0][probs[0]>thresh]=1
            true_classes=np.zeros(1103)
            label_idx=df_data.loc[test_names[img_idx][:-4]].iloc[0].split(" ")
            for lbl_idx in label_idx:
                true_classes[int(lbl_idx)]=1
            FP.append(np.sum((true_classes-probs)==-1))
            FN.append(np.sum((true_classes-probs)==1))
            TP.append(np.sum((true_classes*probs)==1))
            TN.append(1103-FP[-1]-FN[-1]-TP[-1])
        print("for model",i)
        print("TP:",np.mean(TP))
        print("TN:",np.mean(TN))
        print("FP:",np.mean(FP))
        print("FN:",np.mean(FN))

/scratch/1006532/ipykernel_1571117/2002219669.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict=torch.load(state_path, map_location=device)


for model 0.pth
TP: 0.193
TN: 1026.048
FP: 73.807
FN: 2.952
for model 25.0.pth
TP: 0.163
TN: 1031.018
FP: 68.837
FN: 2.982
for model 25.0robust.pth
TP: 1.665
TN: 778.52
FP: 321.335
FN: 1.48
for model 50.0.pth
TP: 0.138
TN: 1050.993
FP: 48.862
FN: 3.007
for model 50.0robust.pth
TP: 2.228
TN: 633.083
FP: 466.772
FN: 0.917
for model 75.0.pth
TP: 0.09
TN: 1087.945
FP: 11.91
FN: 3.055
for model 75.0robust.pth
TP: 3.145
TN: 0.0
FP: 1099.855
FN: 0.0


In [ ]:
state_path=state_dir+"25.0.pth"
state_dict=torch.load(state_path, map_location=device)
model.load_state_dict(state_dict)
model.eval()
image_features = model.encode_image(image_list[0])
text_features = model.encode_text(text)

logits_per_image, logits_per_text = model(image_list[0], text)
probs = logits_per_image.softmax(dim=-1).detach().cpu().numpy()
print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

In [39]:
df_data.loc["1000483014d91860"].iloc[0].split(" ")

['147', '616', '813']

In [51]:
probs[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float16)

In [45]:
a*b

array([ 1,  8, 15])